# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [1]:
from collections import defaultdict
import os
import spacy
from spacy.lang.fr.examples import sentences
!python -m spacy download fr_core_news_md
import pandas as pd

2025-11-04 14:39:06.307063: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-04 14:39:07.936275: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/-fr_core_news_md/-fr_core_news_md.tar.gz#egg===fr_core_news_md contains an egg fragment with a non-PEP 508 name. pip 25.3 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/13157

[notice] A new release of pi

In [10]:
nlp = spacy.load('fr_core_news_md')

## Reconnaissance des entités dans le corpus de 1965

In [11]:
annee_choisie = "1965"
folder_path = r"C:\Users\tommy\TAC2\TAC\data\txt"
corpus_file = os.path.join(folder_path, f"corpus_{annee_choisie}.txt")
output_file = os.path.join(folder_path, f"entites_{annee_choisie}.csv")



# Charger le texte complet de 1965 --> version trop longue !!! et sans sauvegarde !!!
# corpus_file = os.path.join(folder_path, f"corpus_{annee_choisie}.txt")
# with open(corpus_file, 'r', encoding='utf-8') as f:
  #  texte = f.read()

# Analyse linguistique complète du corpus
# nlp.max_length = 3_000_000  # ← augmente la limite à 3 millions de caractères
# doc = nlp(texte)

# Découper le texte en phrases
# sentences = list(doc.sents)

# Parcourir chaque phrase et extraire les entités
# for sent in sentences:
    # entities = [f"{ent.text} ({ent.label_})" for ent in sent.ents]
    # if entities:
        # print(f"→ '{sent.text.strip()}' contient les entités : {', '.join(entities)}")

# refaire en découpant en blocs pour gagner du temps
# === Paramètres ===
# === Charger le modèle spaCy (NER uniquement) ===
nlp = spacy.load("fr_core_news_md", disable=["tagger", "parser", "lemmatizer", "attribute_ruler"])
nlp.max_length = 3_000_000

# === Lire le texte complet ===
with open(corpus_file, 'r', encoding='utf-8') as f:
    texte = f.read()

# === Paramètres de traitement ===
chunk_size = 200_000
entites = []

# === Si le fichier existe déjà, on ne l’écrase pas ===
if not os.path.exists(output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("Texte;Type\n")

# === Boucle par blocs ===
for i in range(0, len(texte), chunk_size):
    bloc = texte[i:i+chunk_size]
    doc = nlp(bloc)
    bloc_ents = [(ent.text, ent.label_) for ent in doc.ents]

    # Sauvegarde immédiate du bloc dans le CSV
    df_bloc = pd.DataFrame(bloc_ents, columns=["Texte", "Type"])
    df_bloc.to_csv(output_file, mode='a', index=False, sep=';', header=False)

    print(f"💾 Bloc {i//chunk_size + 1} traité ({len(bloc):,} caractères, {len(bloc_ents)} entités)")
    entites.extend(bloc_ents)

print(f"\n✅ Analyse terminée : {len(entites):,} entités reconnues.")
print(f"💾 Résultat enregistré dans : {output_file}")

💾 Bloc 1 traité (200,000 caractères, 3899 entités)
💾 Bloc 2 traité (200,000 caractères, 2806 entités)
💾 Bloc 3 traité (200,000 caractères, 3654 entités)
💾 Bloc 4 traité (200,000 caractères, 4579 entités)
💾 Bloc 5 traité (200,000 caractères, 5243 entités)
💾 Bloc 6 traité (200,000 caractères, 5879 entités)
💾 Bloc 7 traité (200,000 caractères, 4056 entités)
💾 Bloc 8 traité (200,000 caractères, 4952 entités)
💾 Bloc 9 traité (200,000 caractères, 5248 entités)
💾 Bloc 10 traité (200,000 caractères, 5564 entités)
💾 Bloc 11 traité (200,000 caractères, 3547 entités)
💾 Bloc 12 traité (200,000 caractères, 3691 entités)
💾 Bloc 13 traité (200,000 caractères, 4940 entités)
💾 Bloc 14 traité (200,000 caractères, 4666 entités)
💾 Bloc 15 traité (109,828 caractères, 1876 entités)

✅ Analyse terminée : 64,600 entités reconnues.
💾 Résultat enregistré dans : C:\Users\tommy\TAC2\TAC\data\txt\entites_1965.csv


## Compter, trier et imprimer les entités du corpus de 1965

In [ ]:
print('doc' in locals())

False


In [ ]:
# Créer un dictionnaire pour chaque type d'entité
people = defaultdict(int)
places = defaultdict(int)
orgs = defaultdict(int)

# Parcourir toutes les entités détectées
for ent in doc.ents:
    label = ent.label_
    text = ent.text.strip()
    if len(text) <= 2:  # filtrer pour sélectionner les mots de minimum 3 caractères
        continue

    if label == "PER":
        people[text] += 1
    elif label == "LOC":
        places[text] += 1
    elif label == "ORG":
        orgs[text] += 1

# Trier les résultats pour chaque type
top_people = Counter(people).most_common(15)
top_places = Counter(places).most_common(15)
top_orgs = Counter(orgs).most_common(15)

# Créer un tableau récapitulatif (DataFrame)
df_summary = pd.DataFrame({
    "Personnes": [p[0] for p in top_people],
    "Fréquence (PER)": [p[1] for p in top_people],
    "Lieux": [p[0] for p in top_places],
    "Fréquence (LOC)": [p[1] for p in top_places],
    "Organisations": [p[0] for p in top_orgs],
    "Fréquence (ORG)": [p[1] for p in top_orgs],
})

# Afficher le tableau
print(df_summary)

NameError: name 'doc' is not defined

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Thérèse apparait 24 fois dans le corpus
Guilbert apparait 22 fois dans le corpus
M. Wilson apparait 20 fois dans le corpus
Rossel apparait 20 fois dans le corpus
Gioconda apparait 18 fois dans le corpus
Molière apparait 17 fois dans le corpus
Enguerrand apparait 17 fois dans le corpus
Wilson apparait 16 fois dans le corpus
Daubrac apparait 15 fois dans le corpus
Parentis apparait 13 fois dans le corpus
Savinien apparait 13 fois dans le corpus
Octave apparait 12 fois dans le corpus
Agence Rossel apparait 12 fois dans le corpus
Monsieur apparait 12 fois dans le corpus
Edwige apparait 12 fois dans le corpus
duc d’Aumale apparait 11 fois dans le corpus
jo no apparait 9 fois dans le corpus
Jules Ferry apparait 9 fois dans le corpus
Naissances apparait 9 fois dans le corpus
Reine apparait 9 fois dans le corpus
Monseigneur apparait 9 fois dans le corpus
Graux apparait 8 fois dans le corpus
Finet apparait 8 fois dans le corpus
Ali-Baba apparait 8 fois dans le corpus
Henri apparait 8 fois dans 

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus